## Final Project Submission

Please fill out:
* Student name: Group 2
* Student pace: sfull time
* Scheduled project review date/time: 10/02/2026
* Instructor name: Samuel.G.Mwangi
* Blog post URL:brian-chairo.hashnode.dev


# Movie Studio Market Analysis
## Stakeholder
Head of the New Movie Studio

## Problem Statement
Our company is launching a new movie studio and needs to decide what types of films to produce in order to maximize box office success. However, the company currently lacks historical knowledge about which movie characteristics lead to strong financial performance.

### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue? By analyzing the total earnings from the different genres, we can determine which types of films are more profitable and likely to attract large audiences.
2. Does movie rating or critic score correlate with higher box office success? using ratings from Rotten Tomatoes and movie database, we can measure whether higher-rated movies earn more revenue 
3. Does a longer runtime affect rating or revenue? By analyzing the runtimes in the dataset, we can identify which runtimes generate higher earnings for different types of movies.
4. Does the release timing affect the earnings? By examining which months and their associated revenue, we can identify the optimal times to launch films for maximum success.

In [6]:
# Unzipping the database in code
import zipfile
import os

zip_path = r"C:\Users\Administrator\Documents\Flatiron\Phase_2\Wk3\Movie-Studio-EDA-Project\zippedData\im.db.zip"
extract_to = r"C:\Users\Administrator\Documents\Flatiron\Phase_2\Wk3\Movie-Studio-EDA-Project\data"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Database extracted!")


Database extracted!


In [7]:
#import libraries
import sqlite3
import pandas as pd

# Path to the database
db_path = r"C:\Users\Administrator\Documents\Flatiron\Phase_2\Wk3\Movie-Studio-EDA-Project\data\im.db"

# Connect to the database
conn = sqlite3.connect(db_path)

#check tables in the database
tables = pd.read_sql("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""", conn)

tables


,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


## Filtering movie_basics table to only remain with what we need 
This step is important as older movies are less relevant, missing runtimes are useless, smaller dataset is faster

In [9]:
# Query Movie_basics table|Filtering only what we need
query_basics = """
SELECT
    movie_id,
    primary_title,
    start_year,
    runtime_minutes,
    genres
FROM movie_basics
WHERE start_year >= 2000
AND runtime_minutes IS NOT NULL
"""
# Load the data
basics = pd.read_sql(query_basics, conn)
basics.head()

,movie_id,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"
4,tt0111414,A Thin Life,2018,75.0,Comedy


## Data quality Filtering movie_ratings table 
Because: votes equalls to 3 or less is not reliable rating thus by filtering we remove the noise

In [10]:
# Query the movie_ratings table to filter out noise
query_ratings = """
SELECT
    movie_id,
    averagerating,
    numvotes
FROM movie_ratings
WHERE numvotes >= 50
"""
# load ratings
ratings = pd.read_sql(query_ratings, conn)
ratings.head()

,movie_id,averagerating,numvotes
0,tt10384606,8.9,559
1,tt1043726,4.2,50352
2,tt1069246,6.2,326
3,tt1094666,7.0,1613
4,tt1130982,6.4,571


In [11]:
#Merge IMDB tables
imdb = basics.merge(ratings, on="movie_id", how="left")
imdb.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",NaN,NaN
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0
4,tt0111414,A Thin Life,2018,75.0,Comedy,NaN,NaN


In [17]:
# load Box Office CSV
box = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
box.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [18]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [20]:
box['foreign_gross'] = pd.to_numeric(
    box['foreign_gross'], errors='coerce'
)
box.dtypes


title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

In [21]:
box['total_gross'] = box['domestic_gross'] + box['foreign_gross']
box['title'] = box['title'].str.strip()

In [25]:
# Load Rotten Tomatoes TSV
rt = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t")

rt['runtime'] = (
    rt['runtime']
    .str.replace(' minutes','', regex=False)
    .astype(float)
)
rt.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104.0,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108.0,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116.0,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128.0,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200.0,NaN


In [24]:
rt.columns

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

In [27]:
#Final Merge
working_df = imdb.merge(
    box,
    left_on="primary_title",
    right_on="title",
    how="left"
)

working_df.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,title,studio,domestic_gross,foreign_gross,year,total_gross
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0111414,A Thin Life,2018,75.0,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
